### Import Required Libraries and Set Up Environment Variables

In [35]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
from pandas import json_normalize
import json

In [36]:
# Set environment variables from the .env in the local environment
load_dotenv("help.env")

nyt_api_key = os.getenv("NY_Times_API")
tmdb_api_key = os.getenv("THEMOVIEDB_API")

### Access the New York Times API

In [37]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
built_url = f'{url}q=&fq={filter_query}&sort={sort}&fl={field_list}&begin_date={begin_date}&end_date={end_date}&api-key={nyt_api_key}'

dumped = requests.get(built_url).json()
print(json.dumps(dumped, indent=4))

{
    "status": "OK",
    "copyright": "Copyright (c) 2024 The New York Times Company. All Rights Reserved.",
    "response": {
        "docs": [
            {
                "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
                "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
                "source": "The New York Times",
                "headline": {
                    "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
                    "kicker": null,
                    "content_kicker": null,
                    "print_headline": "The Attachment Diaries",
                    "name": null,
                    "seo": null,
                    "sub": null
                },
                "keywords": [
                    {
                        "name": "subject",
                        "value": "Movies",
                        "ra

In [38]:
# Create an empty list to store the reviews
review_list = []

# loop through pages 0-19
for page in range(20):
    # create query with a page number
    # API results show 10 articles at a time
    page_number_query = f'{url}q=&fq={filter_query}&page={page}&sort={sort}&fl={field_list}&begin_date={begin_date}&end_date={end_date}&api-key={nyt_api_key}'
    
    # Make a "GET" request and retrieve the JSON
    reviews = requests.get(page_number_query).json()
  

#print(json.dumps(reviews, indent=4))

    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        for doc in reviews["response"]["docs"]:
            review_list.append({
                "web_url": doc["web_url"],
                "snippet": doc["snippet"],
                "source": doc["source"],
                "headline": doc["headline"],
                "keywords": doc["keywords"],
                "pub_date": doc["pub_date"],
                "byline": doc["byline"],
                "word_count": doc["word_count"]})
    except:
        # Print the page number that had no results then break from the loop
        print(f"Page number is out of range.")
        break
        # loop through the reviews["response"]["docs"] and append each review to the list
    
            
        # Print the page that was just retrieved


        # Print the page number that had no results then break from the loop
review_list

[{'web_url': 'https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html',
  'snippet': 'A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.',
  'source': 'The New York Times',
  'headline': {'main': '‘The Attachment Diaries’ Review: Love, Sick',
   'kicker': None,
   'content_kicker': None,
   'print_headline': 'The Attachment Diaries',
   'name': None,
   'seo': None,
   'sub': None},
  'keywords': [{'name': 'subject', 'value': 'Movies', 'rank': 1, 'major': 'N'},
   {'name': 'creative_works',
    'value': 'The Attachment Diaries (Movie)',
    'rank': 2,
    'major': 'N'},
   {'name': 'persons',
    'value': 'Diment, Valentin Javier',
    'rank': 3,
    'major': 'N'}],
  'pub_date': '2023-05-25T11:00:03+0000',
  'byline': {'original': 'By Jeannette Catsoulis',
   'person': [{'firstname': 'Jeannette',
     'middlename': None,
     'lastname': 'Catsoulis',
     'qualifier': None,
     'title': None,
     'rol

In [39]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
first_five = review_list[:5]
json.dumps(first_five, indent=4)


'[\n    {\n        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",\n        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",\n        "source": "The New York Times",\n        "headline": {\n            "main": "\\u2018The Attachment Diaries\\u2019 Review: Love, Sick",\n            "kicker": null,\n            "content_kicker": null,\n            "print_headline": "The Attachment Diaries",\n            "name": null,\n            "seo": null,\n            "sub": null\n        },\n        "keywords": [\n            {\n                "name": "subject",\n                "value": "Movies",\n                "rank": 1,\n                "major": "N"\n            },\n            {\n                "name": "creative_works",\n                "value": "The Attachment Diaries (Movie)",\n                "rank": 2,\n                "major": "N"\n            },\n            {\n     

In [40]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
nyt_reviews_df = json_normalize(review_list)
nyt_reviews_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None


In [41]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
extraction = lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")]
nyt_reviews_df['title'] = nyt_reviews_df['headline.main'].apply(extraction)
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
nyt_reviews_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,The Other Half
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None,The Ottoman Lieutenant
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,Love & Taxes
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",Non

In [42]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
nyt_reviews_df['keywords'] = nyt_reviews_df['keywords'].apply(extract_keywords)

In [43]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles_to_query = nyt_reviews_df['title'].tolist()
titles_to_query

['The Attachment Diaries',
 'What’s Love Got to Do With It?’ Probably a Lo',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley’s Lover',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 'Lingui, the Sacred Bonds',
 'The Pink Clo

### Access The Movie Database API

In [44]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = f"&api_key={tmdb_api_key}"

In [45]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 0

# Loop through the titles
for movie in titles_to_query:
    # Check if we need to sleep before making a request
    if request_counter == 50:
        request_counter = 0
        time.sleep(5)

    # Add 1 to the request counter
    request_counter += 1

    # Perform a "GET" request for The Movie Database
    response = requests.get(f"{url}{movie}{tmdb_key_string}").json()
    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        # Get movie id
        movie_id = response['results'][0]['id']

        # Make a request for a the full movie details
        # Execute "GET" request with url
        movie_details = requests.get(f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}').json()


        # Extract the genre names into a list
        genre_names = []
        for genre in movie_details['genres']:
            genre_names.append(genre['name'])

        # Extract the spoken_languages' English name into a list
        spoken_languages = []
        for language in movie_details['spoken_languages']:
            spoken_languages.append(language['english_name'])

        # Extract the production_countries' name into a list
        production_countries = []
        for country in movie_details['production_countries']:
            production_countries.append(country['name'])

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        movie_data_dict = {
            "title": movie,
            "original_title": movie_details['original_title'],
            "budget": movie_details['budget'],
            "genre": genre_names,
            "language": movie_details['original_language'],
            "spoken_languages": spoken_languages,
            "homepage": movie_details['homepage'],
            "overview": movie_details['overview'],
            "popularity": movie_details['popularity'],
            "runtime(min)": movie_details['runtime'],
            "revenue": movie_details['revenue'],
            "release_date": movie_details['release_date'],
            "vote_average": movie_details['vote_average'],
            "vote_count": movie_details['vote_count'],
            "production_countries": production_countries,
        }

        tmdb_movies_list.append(movie_data_dict)

        # Print out the title that was found
        print(f"Found: {movie_details['title']}")
    except:
        print(f"Not Found: {movie}")
        pass

Found: The Attachment Diaries
Not Found: What’s Love Got to Do With It?’ Probably a Lo
Found: You Can Live Forever
Found: A Tourist's Guide to Love
Found: Other People's Children
Found: One True Loves
Found: The Lost Weekend: A Love Story
Found: A Thousand and One
Found: Your Place or Mine
Found: Love in the Time of Fentanyl
Found: Pamela, A Love Story
Found: In from the Side
Found: After Love
Found: Alcarràs
Found: Nelly Knows Mysteries: A Fatal Engagement
Found: Lady Chatterley's Lover
Found: The Sound of Christmas
Found: The Inspection
Found: Bones and All
Found: My Policeman
Found: About Fate
Found: Waiting for Bojangles
Found: I Love My Dad
Found: A Love Song
Found: Alone Together
Found: Art of Love
Found: 2099: The Soldier Protocol
Found: Thor: Love and Thunder
Found: Both Sides of the Blade
Found: Fire of Love
Found: Mektoub, My Love
Found: Stay Prayed Up
Found: Benediction
Found: Dinner in America
Found: In a New York Minute
Found: Anaïs in Love
Found: I Love America
Found: See

In [46]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
for x in range(5):
    print(json.dumps(tmdb_movies_list[x], indent=4))

{
    "title": "The Attachment Diaries",
    "original_title": "El apego",
    "budget": 0,
    "genre": [
        "Drama",
        "Mystery",
        "Thriller",
        "Horror"
    ],
    "language": "es",
    "spoken_languages": [
        "Spanish"
    ],
    "homepage": "",
    "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
    "popularity": 1.555,
    "runtime(min)": 102,
    "revenue": 0,
    "release_date": "2021-10-07",
    "vote_average": 3.0,
    "vote_count": 4,
    "production_countries": [
        "Argentina"
    ]
}
{
    "title": "You Can Live Forever",
    "original_title": "You Can Live Forever",
    "budget": 0,
    

In [47]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)
tmdb_df.head()

,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime(min),revenue,release_date,vote_average,vote_count,production_countries
0,The Attachment Diaries,El apego,0,"[Drama, Mystery, Thriller, Horror]",es,[Spanish],,"Argentina, 1970s. A desperate young woman goes...",1.555,102,0,2021-10-07,3.000,4,[Argentina]
1,You Can Live Forever,You Can Live Forever,0,"[Drama, Romance]",en,"[English, French]",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",18.161,96,15055,2023-03-24,6.600,40,"[Canada, United States of America]"
2,A Tourist’s Guide to Love,A Tourist's Guide to Love,0,"[Romance, Comedy]",en,"[English, Vietnamese]",https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",18.931,96,0,2023-04-21,6.313,171,[United States of America]
3,Other People’s Children,Les Enfants des autres,0,"[Drama, Comedy]",fr,"[French, English]",https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",9.008,104,84178,2022-09-21,6.813,195,[France]
4,One True Loves,One True Loves,0,"[Romance, Comedy, Drama]",en,"[English, Spanish]",,Emma and Jesse are living the perfect life tog...,14.561,100,37820,2023-04-07,6.458,83,"[Czech Republic, United States of America]"


### Merge and Clean the Data for Export

In [48]:
# Merge the New York Times reviews and TMDB DataFrames on title
df = tmdb_df.merge(nyt_reviews_df, on="title", how="inner")
df.head()

,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime(min),...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,"[Drama, Mystery, Thriller, Horror]",es,[Spanish],,"Argentina, 1970s. A desperate young woman goes...",1.555,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,You Can Live Forever,You Can Live Forever,0,"[Drama, Romance]",en,"[English, French]",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",18.161,96,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
2,A Tourist’s Guide to Love,A Tourist's Guide to Love,0,"[Romance, Comedy]",en,"[English, Vietnamese]",https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",18.931,96,...,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,Other People’s Children,Les Enfants des autres,0,"[Drama, Comedy]",fr,"[French, English]",https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",9.008,104,...,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
4,One True Loves,One True Loves,0,"[Romance, Comedy, Drama]",en,"[English, Spanish]",,Emma and Jesse are living the perfect life tog...,14.561,100,...,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None


In [49]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
fix_columns_list = ["genre", "spoken_languages", "production_countries"]

# Create a list of characters to remove
chars_to_remove = ["[", "]", '"', "'"]

# Loop through the list of columns to fix
for column in fix_columns_list:

    # Convert the column to type 'str'
    df[column] = df[column].astype(str)

    # Loop through characters to remove
    for char in chars_to_remove:
        df[column] = df[column].str.replace(char, '', regex=False)

# Display the fixed DataFrame
df.head()

,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime(min),...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",1.555,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,You Can Live Forever,You Can Live Forever,0,"Drama, Romance",en,"English, French",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",18.161,96,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
2,A Tourist’s Guide to Love,A Tourist's Guide to Love,0,"Romance, Comedy",en,"English, Vietnamese",https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",18.931,96,...,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,Other People’s Children,Les Enfants des autres,0,"Drama, Comedy",fr,"French, English",https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",9.008,104,...,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
4,One True Loves,One True Loves,0,"Romance, Comedy, Drama",en,"English, Spanish",,Emma and Jesse are living the perfect life tog...,14.561,100,...,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None


In [50]:
# Drop "byline.person" column
df.drop(columns=['byline.person'], inplace=True)

In [51]:
# Delete duplicate rows and reset index
df = df.drop_duplicates(subset='title', keep='first').reset_index(drop=True)
df.head()

,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime(min),...,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",1.555,102,...,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None
1,You Can Live Forever,You Can Live Forever,0,"Drama, Romance",en,"English, French",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",18.161,96,...,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None
2,A Tourist’s Guide to Love,A Tourist's Guide to Love,0,"Romance, Comedy",en,"English, Vietnamese",https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",18.931,96,...,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,None
3,Other People’s Children,Les Enfants des autres,0,"Drama, Comedy",fr,"French, English",https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",9.008,104,...,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,None
4,One True Loves,One True Loves,0,"Romance, Comedy, Drama",en,"English, Spanish",,Emma and Jesse are living the perfect life tog...,14.561,100,...,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,None


In [53]:
# Export data to CSV without the index
df.to_csv('Module_6', index=False)